In [1]:
import json
import torch
from pathlib import Path
from typing import List
from IPython.display import clear_output

## Preparation

In [2]:
IN_JSONL             = ... # path to generations
OUT_JSONL            = "judge_curated_model.jsonl"
ANSWERS_LOG_JSONL    = "answers_log_model.jsonl"
STATE_JSON           = "judge_state_model.json"


answers = torch.load(IN_JSONL, map_location="cpu")

In [3]:
answers[0].keys()  # take a look at keys

dict_keys(['idx', 'model', 'tokenizer', 'prompt_len', 'output_len', 'output_text', 'output_token_strings', 'seq', 'question', 'hiddens', 'split_idx'])

In [4]:
len(set(ans["idx"] for ans in answers))

501

#### IMPORTANT

Insert your split index here:

In [5]:
SPLIT_INDEX = 0 # we split dataset across several people 

answers = [
    ans for ans in answers if ans["split_idx"] == SPLIT_INDEX
]
print(f"Loaded split with index {SPLIT_INDEX}")
assert len(answers) >= 125, f"Something wrong with the split, check `SPLIT_INDEX` (Needs to be 0/1/2/3) variable, Need at least 125 questions, got: {len(answers)}"
assert all([len(x["output_token_strings"]) == len(x["hiddens"][0]) for x in answers]), "Number of hiddens and numfer of output indices mismatch"
print(f"Number of generations to annotate: {len(set(ans['idx'] for ans in answers))}")

Loaded split with index 1
Number of generations to annotate: 125


In [6]:
def sha1(s: str) -> str:
    import hashlib as _hl
    return _hl.sha1(s.encode("utf-8")).hexdigest()[:10]

def pretty_print_tokens(tokens: List[str]) -> None:
    print("Draft tokens (index : token):")
    for i, t in enumerate(tokens):
        disp = t.replace("Ġ","␠").replace("▁","␠")
        print(f"{i:>4}: {disp}")



# Resume: read state + what is already in OUT_JSONL
def load_state():
    if Path(STATE_JSON).exists():
        try:
            return json.loads(Path(STATE_JSON).read_text())
        except Exception:
            pass
    return {"next_index": 0}

def save_state(next_index: int):
    Path(STATE_JSON).write_text(json.dumps({"next_index": next_index}))

In [7]:
processed_prompts = set()
if Path(OUT_JSONL).exists():
    try:
        with open(OUT_JSONL, "r", encoding="utf-8") as f:
            for line in f:
                obj = json.loads(line)
                if "prompt_sha" in obj:
                    processed_prompts.add(obj["prompt_sha"])
    except Exception:
        pass

state = load_state()
start_idx = max(0, int(state.get("next_index", 0)))

## Manual annotation (TASK START)

- You will see the question and model's answer, read it and try to identify the index at which the model starts generating wrong answer. If whole answer seems correct, send `-1`
- If you want to exit annotation and come back earlier, write `BREAK` in the input prompt

In [8]:
# ================= LOOP =================
written_rows = 0
selected = start_idx

for ds_idx in range(start_idx, len(answers)):
    ex = answers[ds_idx]
    ans = ex["output_text"]
    prompt = ex['question']
    prompt_sha = sha1(prompt)

    prim_tokens = ex["output_token_strings"]
    if prompt_sha in processed_prompts:
        continue

    clear_output()

    # Show tokens + indices for you to pick the first wrong token
    print(f"========= [{selected+1}/{len(answers)}] QUESTION ======:\n{prompt}")
    print(f"========= ANSWER ========:\n{ans}\n======= TOKENS ======= \n")
    pretty_print_tokens(prim_tokens)

    # --- Get the error index from user ---
    break_ = False
    while True:
        try:
            x = input("\nEnter the index of the FIRST WRONG token (0..N-1). "
                                "If entirely correct, type -1:\n> ").strip()
            if x == "BREAK":
                raise KeyboardInterrupt
            err_idx = int(x)
            if err_idx >= len(prim_tokens) and err_idx != -1:
                raise ValueError
            break
        except KeyboardInterrupt:
            break_ = True
            break
        except Exception:
            print("Invalid index. Try again.")
    if break_:
        break

    if err_idx == -1:
        err_idx = len(prim_tokens)  # all tokens are correct

    Path(OUT_JSONL).touch(exist_ok=True)
    Path(ANSWERS_LOG_JSONL).touch(exist_ok=True)

    total_rows = 0
    with open(OUT_JSONL, "a", encoding="utf-8") as f:
        recording = dict(
            prompt=prompt,
            answer_text=ans,
            prompt_sha=prompt_sha,
            source=ex["model"],
            hiddens=[i.tolist() for i in ex["hiddens"][0]],
            changed_token_indices=[(i, i < err_idx) for i in range(len(ex["output_token_strings"]))],
        )
        f.write(json.dumps(recording, ensure_ascii=False) + "\n")

        total_rows = len(ex["output_token_strings"])

    with open(ANSWERS_LOG_JSONL, "a", encoding="utf-8") as g:
        g.write(json.dumps({
            "prompt_sha": prompt_sha,
            "prompt_text": prompt,
            "error_start_index": err_idx
        }, ensure_ascii=False) + "\n")

    written_rows += total_rows
    processed_prompts.add(prompt_sha)
    selected += 1

    save_state(ds_idx + 1)

print(f"\nDone. Wrote {written_rows} rows across {selected} example(s).")
print(f"Resume pointer saved to {STATE_JSON}. You can rerun to continue from the next unfinished prompt.")

========= [63/125] QUESTION ======:
system

Cutting Knowledge Date: December 2023
Today Date: 17 Oct 2025

user

### Instruction:
Generate a valid HTML code for a box of 500x500 pixel sizeassistant


========= ANSWER ========:
### HTML Code for a 500x500 Pixel Box

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>500x500 Pixel Box</title>
    <style>
       .box {
            width: 500px;
            height: 500px;
            border: 1px solid black;
            background-color: #f0f0f0;
            padding: 10px;
            box-sizing: border-box;
        }
    </style>
</head>
<body>
    <div class="box">
        <!-- You can add content here -->
    </div>
</body>
</html>
```

### Explanation:

*   We start with the basic HTML structure, including the `DOCTYPE`, `html`, `head`, and `body` tags.
*   In the `head` section, we define a `style` block to apply CSS styles to

## Send the results


Send us the file saved at `OUT_JSONL`